In [1]:
import pandas as pd
import numpy as np
import os

## 1 - Load the preprocessed data

In [2]:
processed_file_path = os.path.join('..', 'data', 'processed')
clustering_model_name = 'KMeans'
processed_file_name = f'train_{clustering_model_name}.csv'

train_df = pd.read_csv(os.path.join(processed_file_path, processed_file_name), low_memory=False)
train_df.head()

,file,VMONTH,VYEAR,VDAYR,YEAR,AGE,SEX,ETHNIC,RACE,USETOBAC,...,OTHPROV,MHP,NODISP,REFOTHMD,RETAPPT,OTHDISP,ERADMHOS,cluster,CombinedText,ProcessedText
0,opd2006.csv,December,2006.0,Friday,2006.0,55.0,Male,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,9,55_year_old Middle_Aged Male Acute problem Inj...,55_year_old middle_age male acute problem inju...
1,opd2006.csv,November,2006.0,Thursday,2006.0,66.0,Male,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,0,66_year_old Senior Male Acute problem Cough Ov...,66_year_old senior male acute problem cough ov...
2,opd2006.csv,November,2006.0,Wednesday,2006.0,71.0,Female,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,Yes,No,No,No,7,71_year_old Senior Female Acute problem Genera...,71_year_old senior female acute problem genera...
3,opd2006.csv,November,2006.0,Tuesday,2006.0,1.0,Female,Not Hispanic or Latino,White Only,Not current,...,No,NaN,One or more dispositions marked,No,No,No,No,2,1_year_old Infant Female Acute problem Other a...,1_year_old infant female acute problem unspeci...
4,opd2006.csv,November,2006.0,Monday,2006.0,21.0,Female,Not Hispanic or Latino,White Only,Current,...,No,NaN,One or more dispositions marked,No,No,No,No,6,21_year_old Adult Female TOBACCO user Acute pr...,21_year_old adult female tobacco user acute pr...


## 2 - Topic modeling

In [3]:
import gensim
from gensim import corpora

random_seed = 42

In [4]:
# Generate bigrams
def generate_bigrams(row):
    tokens = row.split()
    bigram_model = gensim.models.Phrases(tokens, min_count=5, threshold=100)
    tokens = [token for token in bigram_model[tokens]]
    return tokens


print(generate_bigrams(train_df['ProcessedText'].iloc[0]))
preprocessed_docs = train_df['ProcessedText'].apply(generate_bigrams)

['55_year_old', 'middle_age', 'male', 'acute', 'problem', 'injury', 'unspecified', 'sho', 'obesity', 'normal_temperature', 'hypertension', 'high_diastolic_blood_pressure', 'diabetes', 'hyperlipidemia', 'obesity', 'currently', 'enrol', 'disease', 'management', 'program', 'shoulder', 'upper', 'arm', 'injury']


In [5]:
# Build dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_docs)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]

In [6]:
# Train LDA model
n_topics = 10

lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=n_topics,
    random_state=random_seed,
    chunksize=100,
    passes=10,
)



### 2.1 - Visualize the topics

In [10]:
import pyLDAvis.gensim_models
import pyLDAvis

pyLDAvis.enable_notebook()

In [12]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.192250 -0.039214       1        1  19.993371
2     -0.178751 -0.147501       2        1  11.243633
6      0.143707 -0.039789       3        1  11.201200
8     -0.099027  0.008373       4        1  10.668214
5     -0.230448 -0.070492       5        1   9.792321
0     -0.039568  0.131800       6        1   9.719827
1      0.132057  0.074250       7        1   9.076431
7     -0.098304  0.029713       8        1   8.201354
9      0.166012 -0.200285       9        1   5.209086
4      0.012072  0.253144      10        1   4.894563, topic_info=                    Term          Freq         Total Category  logprob  \
461                onset  20697.000000  20697.000000  Default  30.0000   
1065                   3  20546.000000  20546.000000  Default  29.0000   
3601                 mos  20545.000000  20545.000000  Default  28.0000   
3602                 new  20545.000000  20545.000000  Default  27.0000   
5                disease  34021.000000  34021.000000  Default  26.0000   
...                  ...           ...           ...      ...      ...   
72                   nos   1607.635781  19593.520650  Topic10  -3.9899   
1                  acute    980.730683   3454.838384  Topic10  -4.4841   
14    normal_temperature   1118.335151  43223.912142  Topic10  -4.3529   
41                female   1103.242012  67183.561134  Topic10  -4.3664   
9           hypertension    890.807370  40306.336193  Topic10  -4.5803   

      loglift  
461   30.0000  
1065  29.0000  
3601  28.0000  
3602  27.0000  
5     26.0000  
...       ...  
72     0.5166  
1      1.7578  
14    -0.6375  
41    -1.0921  
9     -0.7951  

[605 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
264       4  0.188832  10_year_old
264       5  0.810044  10_year_old
617       5  0.100436  13_year_old
617       8  0.899101  13_year_old
363       5  0.185517  16_year_old
...     ...       ...          ...
209       1  0.894734       weight
209       5  0.103306       weight
350       1  0.994929     wheezing
355       6  0.998310        wrist
2066      2  0.997025   xerography

[776 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 7, 9, 6, 1, 2, 8, 10, 5])

## 3 - Get the topic distribution for each cluster

In [13]:
# Get the topic distribution for each document 
def get_avg_topic_distribution(cluster_data, topic_distribution, num_topics):
    cluster_topics = np.zeros((len(cluster_data.index), num_topics))

    # Update the distribution with the actual values
    for i, doc_index in enumerate(cluster_data.index.tolist()):
        for topic, prob in topic_distribution[doc_index]:
            cluster_topics[i, topic] = prob

    avg_topic_dist = np.mean(cluster_topics, axis=0)
    return avg_topic_dist


topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0.0)
clusters = train_df.groupby('cluster').ProcessedText

cluster_topics = []
for cluster, data in clusters:
    avg_topic_dist = get_avg_topic_distribution(data, topic_distribution, lda_model.num_topics)
    cluster_topics.append([avg_topic_dist.tolist()])

cluster_topics_df = pd.DataFrame(cluster_topics, columns=['avg_topic_distribution'])

cluster_topics_df

,avg_topic_distribution
0,"[0.07912902421327017, 0.10313202121848415, 0.0..."
1,"[0.0448975058168897, 0.026481083959176806, 0.5..."
2,"[0.05127262405342381, 0.02201387012689692, 0.0..."
3,"[0.2766385119875507, 0.05624676887809734, 0.10..."
4,"[0.06986006324503005, 0.1298441667637168, 0.05..."
5,"[0.1622810542547156, 0.11672142382313729, 0.11..."
6,"[0.15280885904836086, 0.05469147632924766, 0.0..."
7,"[0.0783366400262008, 0.11394882010759393, 0.04..."
8,"[0.044404195558600604, 0.021014036628259302, 0..."
9,"[0.08893814093648142, 0.08788535734928002, 0.1..."


### 3.2 - Visualize the distribution of topics within each cluster

In [14]:
import altair as alt

In [15]:
# Reshape the dataframe to have a row for each cluster and topic
cluster_topics_heatmap_df = cluster_topics_df.avg_topic_distribution.apply(pd.Series).reset_index().rename(columns={'index': 'cluster'}).melt(id_vars='cluster', var_name='topic', value_name='probability')

heatmap = alt.Chart(cluster_topics_heatmap_df).mark_rect().encode(
    x='topic:O',
    y='cluster:O',
    color='probability:Q'
).properties(
    title='Average Topic Distribution for Each Cluster',
    width=400,
    height=400
)

heatmap

alt.Chart(...)